In [1]:
%%bash
chia version
cdv --version
python --version

1.6.1rc3.dev49
cdv, version 1.1.2
Python 3.10.8


In [ ]:
%%bash
CUR_HEIGHT=$(chia rpc full_node get_blockchain_state | jq ".blockchain_state.peak.height")
echo $CUR_HEIGHT
echo $((CUR_HEIGHT-4))
cdv sim -n announcement revert -fd --blocks $((CUR_HEIGHT-4))

## Run on simulator
```sh
❯ chia show -s
Network: simulator0    Port: 43941   RPC Port: 23058
Node ID: 85c082f243d0fc97428c827e62bfa8c48ed1a181a0a3b399bf44f67e508b99e3
Genesis Challenge: eb8c4d20b322be8d9fddbf9412016bdffe9a2901d7edb0e364e94266d0e095f7
Current Blockchain Status: Full Node Synced

Peak: Hash: 93eedac96ec5b56db0dc1938cb088b474b53c9c07229480265150a0132b17408
      Time: Sun Oct 30 2022 15:11:05 +07                  Height:          4

Estimated network space: 51.785 MiB
Current difficulty: 1024
Current VDF sub_slot_iters: 1024

  Height: |   Hash:
        4 | 93eedac96ec5b56db0dc1938cb088b474b53c9c07229480265150a0132b17408
        3 | 300661e1d55817018f30d9e36b3f68959e0b89ab0d3981d1667da9aabd6737ba
        2 | 21c8b860aef6ba7cf5445b5bc1c60fd43a000f3465a42b41f438d6a047d0149b
        1 | 390ac4b8ae997289f4cdee933d870d9d12b529ac83c925a7f5d02eeb25a69a88
```

## Alice's Coin Announcement

In [ ]:
%%bash
chia keys show --json | jq

In [ ]:
%%bash
cdv decode txch1rcr6pusjdvagw53c8hxgsg9m6qxd3s7jdyhusq2c788g7d69xupse7js8m

In [10]:
# chia libraries
from blspy import (PrivateKey, AugSchemeMPL, G1Element, G2Element)

from chia.consensus.default_constants import DEFAULT_CONSTANTS
from chia.types.blockchain_format.coin import Coin
from chia.types.blockchain_format.program import Program
from chia.types.coin_spend import CoinSpend
from chia.types.condition_opcodes import ConditionOpcode
from chia.types.spend_bundle import SpendBundle
from chia.util.hash import std_hash
from chia.wallet.puzzles import p2_delegated_puzzle_or_hidden_puzzle

from chia.wallet.puzzles.p2_delegated_puzzle_or_hidden_puzzle import (
    DEFAULT_HIDDEN_PUZZLE_HASH,
    calculate_synthetic_secret_key,
    puzzle_for_pk,
    puzzle_for_conditions,
    solution_for_conditions,
)

from clvm.casts import int_to_bytes
from clvm_tools.clvmc import compile_clvm_text
from clvm_tools.binutils import disassemble

# utils & tic tac toe helper code
import sys
sys.path.insert(0, "../../../shared")
from utils import (load_program, print_program, print_puzzle, print_json, print_push_tx_result)


In [11]:
from chia.rpc.full_node_rpc_client import FullNodeRpcClient
from chia.util.config import load_config
from chia.util.default_root import DEFAULT_ROOT_PATH
from chia.util.ints import uint16

config = load_config(DEFAULT_ROOT_PATH, "config.yaml")
selected_network = config["selected_network"]
genesis_challenge = config["network_overrides"]["constants"][selected_network]["GENESIS_CHALLENGE"]

self_hostname = config["self_hostname"] # localhost
full_node_rpc_port = config["full_node"]["rpc_port"] # 8555

```sh
❯ chia keys derive -f $fp_alice wallet-address -i 0 --show-hd-path
Wallet address 0 (m/12381/8444/2/0): txch1rcr6pusjdvagw53c8hxgsg9m6qxd3s7jdyhusq2c788g7d69xupse7js8m

❯ cdv decode txch1rcr6pusjdvagw53c8hxgsg9m6qxd3s7jdyhusq2c788g7d69xupse7js8m
1e07a0f2126b3a8752383dcc8820bbd00cd8c3d2692fc80158f1ce8f37453703

~
❯ chia keys derive -f $fp_alice child-key -t wallet -i 0 --show-hd-path -s
Wallet public key 0 (m/12381/8444/2/0): b92ff51129c3e792edee9144168902567fa8e595b2196e4a654614ab024d31de42dda7ca243a65b6e11b5664c8f58947
Wallet private key 0 (m/12381/8444/2/0): 4088622ebb45d037bb313695e798a803597bdfc3fcd9151cd7fee3fd92feec4e
```

In [16]:
alice_sk = PrivateKey.from_bytes(bytes.fromhex("4088622ebb45d037bb313695e798a803597bdfc3fcd9151cd7fee3fd92feec4e"))
alice_pk = alice_sk.get_g1()
alice_ph = bytes.fromhex("1e07a0f2126b3a8752383dcc8820bbd00cd8c3d2692fc80158f1ce8f37453703")
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
coin_records = await full_node_client.get_coin_records_by_puzzle_hash(alice_ph)
full_node_client.close()
await full_node_client.await_closed()

alice_coin = coin_records[0].coin
print_json(alice_coin.to_json_dict())

{
    "amount": 1750000000000,
    "parent_coin_info": "0xeb8c4d20b322be8d9fddbf9412016bdf00000000000000000000000000000002",
    "puzzle_hash": "0x1e07a0f2126b3a8752383dcc8820bbd00cd8c3d2692fc80158f1ce8f37453703"
}


In [17]:
message = bytes("hello", "utf-8")
announcement = std_hash(message)
alice_conditions = [
    [
        ConditionOpcode.CREATE_COIN_ANNOUNCEMENT,
        announcement
    ],

    # return
    [
        ConditionOpcode.CREATE_COIN, 
        alice_coin.puzzle_hash, 
        alice_coin.amount
    ]
]

puzzle_reveal = puzzle_for_pk(alice_pk)
delegated_puzzle: Program = puzzle_for_conditions(alice_conditions) 
solution = solution_for_conditions(alice_conditions)

alice_coin_spend = CoinSpend(
    alice_coin,
    puzzle_reveal,
    solution
)
    
synthetic_sk: PrivateKey = calculate_synthetic_secret_key(
    alice_sk,
    DEFAULT_HIDDEN_PUZZLE_HASH
)

alice_sig = AugSchemeMPL.sign(synthetic_sk,
    (
        delegated_puzzle.get_tree_hash()
        + alice_coin.name()
        + DEFAULT_CONSTANTS.AGG_SIG_ME_ADDITIONAL_DATA
    )
)

spend_bundle = SpendBundle(
    [alice_coin_spend],
    alice_sig
)
print_json(spend_bundle.to_json_dict())

{
    "aggregated_signature": "0x89ac850a8b3db3d29bfaaba81a89e36714e8fe07b4bbf937ecb34d3364f1915fdeebe95a36462626afb3ec54f145a0e102fffbdb9a3b002cf6eb9ab5ba685f7e24ba2613de37033969069881193a83dd6b93b5c73d6a87743070428b5a3c7635",
    "coin_solutions": [
        {
            "coin": {
                "amount": 1750000000000,
                "parent_coin_info": "0xeb8c4d20b322be8d9fddbf9412016bdf00000000000000000000000000000002",
                "puzzle_hash": "0x1e07a0f2126b3a8752383dcc8820bbd00cd8c3d2692fc80158f1ce8f37453703"
            },
            "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff80808080

### Bob's
```sh
❯ chia keys derive -f $fp_bob wallet-address -i 0 --show-hd-path
Wallet address 0 (m/12381/8444/2/0): txch1adsehp8swwmhzrmq0ctucja05070fumzq88hppt972m85aczy9hszmy3s8

~
❯ cdv decode txch1adsehp8swwmhzrmq0ctucja05070fumzq88hppt972m85aczy9hszmy3s8
eb619b84f073b7710f607e17cc4bafa3fcf4f36201cf708565f2b67a7702216f

~
❯ chia keys derive -f $fp_bob child-key -t wallet -i 0 --show-hd-path -s
Wallet public key 0 (m/12381/8444/2/0): 8ee201eda461ccf142cdd3042cc22a2c1efba8bbd13ccb51602dfdf4074f33240c7b0418c209cca51352162d77dab249
Wallet private key 0 (m/12381/8444/2/0): 59baf8f28b17d616e5737e700beefae3f5bb2be927708bdd68066c2d992da77b
```

In [13]:
bob_sk = PrivateKey.from_bytes(bytes.fromhex("59baf8f28b17d616e5737e700beefae3f5bb2be927708bdd68066c2d992da77b"))
bob_pk = bob_sk.get_g1()
bob_ph = bytes.fromhex("eb619b84f073b7710f607e17cc4bafa3fcf4f36201cf708565f2b67a7702216f")
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
coin_records = await full_node_client.get_coin_records_by_puzzle_hash(bob_ph)
full_node_client.close()
await full_node_client.await_closed()

bob_coin = coin_records[0].coin
print_json(bob_coin.to_json_dict())

{
    "amount": 250000000000,
    "parent_coin_info": "0xfe9a2901d7edb0e364e94266d0e095f700000000000000000000000000000003",
    "puzzle_hash": "0xeb619b84f073b7710f607e17cc4bafa3fcf4f36201cf708565f2b67a7702216f"
}


In [14]:
bob_conditions = [
    [
        ConditionOpcode.ASSERT_COIN_ANNOUNCEMENT,
        std_hash(alice_coin.name() + announcement)
    ],
    
    # return
    [
        ConditionOpcode.CREATE_COIN, 
        bob_coin.puzzle_hash, 
        bob_coin.amount
    ]
]

puzzle_reveal = puzzle_for_pk(bob_pk)
delegated_puzzle: Program = puzzle_for_conditions(bob_conditions) 
solution = solution_for_conditions(bob_conditions)

bob_coin_spend = CoinSpend(
    bob_coin,
    puzzle_reveal,
    solution
)
    
synthetic_sk: PrivateKey = calculate_synthetic_secret_key(
    bob_sk,
    DEFAULT_HIDDEN_PUZZLE_HASH
)

bob_sig = AugSchemeMPL.sign(synthetic_sk,
    (
        delegated_puzzle.get_tree_hash()
        + bob_coin.name()
        + DEFAULT_CONSTANTS.AGG_SIG_ME_ADDITIONAL_DATA
    )
)

spend_bundle = SpendBundle(
    [bob_coin_spend],
    bob_sig
)
print_json(spend_bundle.to_json_dict())

{
    "aggregated_signature": "0xae373df0c410dc18a76ba9312874243599e6e0b9230bc9b80cc2ba71971cc01b6a489c2f175eeb65957c1297c111ca300e215e46f2a7c60fb09cc508d5fd46f6a879f7830c781e4e880753ef8ad231750c473b793a359f36050f69d23008d015",
    "coin_solutions": [
        {
            "coin": {
                "amount": 250000000000,
                "parent_coin_info": "0xfe9a2901d7edb0e364e94266d0e095f700000000000000000000000000000003",
                "puzzle_hash": "0xeb619b84f073b7710f607e17cc4bafa3fcf4f36201cf708565f2b67a7702216f"
            },
            "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff808080808

In [15]:
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
result = await full_node_client.push_tx(spend_bundle)
full_node_client.close()
await full_node_client.await_closed()
print(result)

ValueError: {'error': 'Failed to include transaction 9426414b7370e21d802fc33241fbe614a0263b21a3a1603748c7cb3f5107eddc, error ASSERT_ANNOUNCE_CONSUMED_FAILED', 'success': False}